In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 91.7 ms


In [3]:
#adding this in fow now, will eventually want to switch this to the 03-clinical_variables_final
import psycopg2
import getpass
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 2.32 s


#### Note: Nelson has said we do not need to incorporate Echo data, so this has been removed 10/22/18

In [34]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018'  #date of the files i will be importing

ventcategory_df=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date)), index_col=0)

vaso_dose_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date)), index_col=0)


# echodata_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date)), index_col=0)

labs_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)


weightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date)), index_col=0)
heightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date)), index_col=0)

vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)
uti_all_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date)), index_col=0)
bg_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)

rrt_merged_allpt_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date)), index_col=0)
gcs72_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date)), index_col=0)

sofa_df_72=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 13.3 s


In [5]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
    return(df)

time: 4.62 ms


In [6]:
# subsetting each dataframe to only patients in final_patients_df2_v:
dataframe_list= [
    ventcategory_df, vaso_dose_72, #echodata_72,
    labs_all_nosummary_72, weightfirstday_df,
    heightfirstday_df, vitals_all_nosummary_72,
    uti_all_72, bg_all_nosummary_72,
    rrt_merged_allpt_df, gcs72_df, sofa_df_72
]

(
ventcategory_df, vaso_dose_72, #echodata_72,
labs_all_nosummary_72, weightfirstday_df,
heightfirstday_df, vitals_all_nosummary_72,
uti_all_72, bg_all_nosummary_72,
rrt_merged_allpt_df, gcs72_df, sofa_df_72
) = (df_subset(df) for df in dataframe_list)

time: 1 s


In [7]:
# date that i ran the below code on the files.

date= '05112018'

time: 814 µs


## vaso dose

In [8]:
#vaso_dose_72.head()
vaso_dose_72= vaso_dose_72.rename(
    columns={'starttime':'charttime','label':'vaso_type'}
        )#.drop(labels='endtime', axis=1) 
#renaming starttime to charttime and dropping endtime

time: 22.6 ms


In [9]:
len(vaso_dose_72)

53786

time: 3.15 ms


In [10]:
vaso_dose_72.head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
92,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
93,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,1.962759,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 12:00:00.000000000
94,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,2.000000,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 21:59:00.000000000
106,200952.0,2139-09-23 23:12:00,2139-09-24 21:54:00,0.149992,7.990853,mg,mcg/kg/min,norepinephrine,2139-09-23,0 days 23:12:00.000000000


time: 16.1 ms


In [11]:
#removing units/hour because that is a different use of vasopressin
vaso_dose_72 = vaso_dose_72.loc[vaso_dose_72.loc[:,'rate_uom']!= 'units/hour',:]
vaso_dose_72 = vaso_dose_72.loc[vaso_dose_72.loc[:,'rate_uom']!= 'Uhr',:]

time: 17 ms


In [12]:
len(vaso_dose_72)

52976

time: 1.84 ms


In [13]:
vaso_dose_72['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 4.44 ms


#### removing outliers/extreme values

In [14]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:

test_group=(vaso_dose_72[['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
     .groupby(['vaso_type','rate_uom'])
         )

vaso_dose_72['std']=test_group.transform(lambda x : x.std())
vaso_dose_72['mean']=test_group.transform(lambda x : x.mean())

normal_high_value= pd.DataFrame({
    'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
    'high_value': [40, 20, 0.5, 1, 0.1, 2]
}) #found from literature, see notes

vaso_dose_72 = pd.merge(vaso_dose_72, normal_high_value, left_on='vaso_type', right_on='vaso_type')

vaso_dose_72_rmout =(vaso_dose_72[
    ~((vaso_dose_72['vaso_rate'] > vaso_dose_72['high_value']) & ((vaso_dose_72['vaso_rate']-vaso_dose_72['mean'])>= (3*vaso_dose_72['std'])))
    ])
#ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean

time: 161 ms


In [15]:
len(vaso_dose_72) #52976

52976

time: 1.91 ms


In [16]:
vaso_dose_72['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 3.68 ms


In [17]:
print(len(vaso_dose_72_rmout), len(vaso_dose_72)) #works, only excluded ~300/52662 for outliers. 


52662 52976
time: 928 µs


In [18]:
vaso_dose_72_rmout['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 3.9 ms


In [19]:
#date= '22102018'
pd.DataFrame(vaso_dose_72_rmout).to_csv(Path(
    wd+'/data/processed/%s_vaso_prepped.csv' %(date)))


time: 568 ms


## vitals -

In [188]:
vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 9.9 s


In [189]:
vitals_all_nosummary_72 = vitals_all_nosummary_72.rename(index=str, columns={"valueuom":"uom","vitalid":'label', 'valuenum':'value'}) #change valueom to uom
vitals_all_nosummary_nonull = vitals_all_nosummary_72.loc[vitals_all_nosummary_72['label'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull.loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='RespRate') & 
    (vitals_all_nosummary_nonull['uom']=='bpm'),'uom']='insp/min'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempC') & 
    (vitals_all_nosummary_nonull['uom']=='?C'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempF') & 
    (vitals_all_nosummary_nonull['uom']=='Deg. F'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempF') & 
    (vitals_all_nosummary_nonull['uom']=='?F'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempF'),'label']='TempC'

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


time: 14.5 s


- glucose max
- glucose min
- diasBP min
- heartrate min
- meanart pressure min
- RespRate min
- SYSbp min
- TEMPC min

In [181]:
#writing a function to aggregate for time length

def timewindow_agg(df, label='', roundwindow= '120min', value_col='value', transform_fxn=max):
    df['rounded_time']=pd.to_datetime(df['charttime']).dt.round(roundwindow)
    subset_df=(df.loc[df.loc[:,'label']==label, :])

    #test_df4.groupby(['icustay_id','rounded_time'])['value'].apply(lambda x: max(x)).sort_values() #works!test_df4
    #getting the rows for the max value for each rounded_time. 
    idx= subset_df.groupby(['icustay_id','rounded_time'])['value'].transform(transform_fxn) ==subset_df['value']
    (df.loc[df.loc[:,'label']==label, :])= subset_df[idx]
    return(df)

time: 15.9 ms


In [190]:
#glucose
(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'icustay_id']==228393.0) &
                         (vitals_all_nosummary_nonull.loc[:,'label']=='Glucose') &
                         (vitals_all_nosummary_nonull.loc[:,'value']>99999), 'value'])=np.nan

time: 525 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [23]:
# pd.DataFrame(vitals_all_nosummary_nonull).to_csv(Path(
#     wd+'/data/processed/%s_vitals_prepped.csv' %(date)))


time: 792 µs


In [192]:
#aggregating vital readings to max value over 3 hours. 
timewindow_agg(vitals_all_nosummary_nonull, label='Glucose', roundwindow= '180min', value_col='value', transform_fxn=max)

timewindow_agg(vitals_all_nosummary_nonull, label='DiasBP', roundwindow= '180min', value_col='value', transform_fxn=max)

timewindow_agg(vitals_all_nosummary_nonull, label='HeartRate', roundwindow= '180min', value_col='value', transform_fxn=max)

timewindow_agg(vitals_all_nosummary_nonull, label='MeanArtPress', roundwindow= '180min', value_col='value', transform_fxn=max)

timewindow_agg(vitals_all_nosummary_nonull, label='RespRate', roundwindow= '180min', value_col='value', transform_fxn=max)

timewindow_agg(vitals_all_nosummary_nonull, label='SysBP', roundwindow= '180min', value_col='value', transform_fxn=max)

timewindow_agg(vitals_all_nosummary_nonull, label='TempC', roundwindow= '180min', value_col='value', transform_fxn=max)


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,subject_id,hadm_id,icustay_id,charttime,uom,label,value,t_0,delta,rounded_time
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7,9.0,150750.0,220597.0,2149-11-14 12:00:00,mmHg,MeanArtPress,134.0,2149-11-13,1 days 12:00:00.000000000,2149-11-14 12:00:00
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
13,9.0,150750.0,220597.0,2149-11-14 02:00:00,mmHg,MeanArtPress,96.0,2149-11-13,1 days 02:00:00.000000000,2149-11-14 03:00:00
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
18,9.0,150750.0,220597.0,2149-11-13 00:30:00,mmHg,MeanArtPress,104.0,2149-11-13,0 days 00:30:00.000000000,2149-11-13 00:00:00
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


time: 55.4 s


In [184]:
vitals_all_nosummary_nonull = vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull['value'].notnull(),:]#.count() #6930 NULL values


time: 418 ms


In [191]:
#distribution after no window aggregate
(vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'value'].notna(), ['label','value']
]
     .groupby(['label'])
         .describe(percentiles=[.25, .5, .75,.95, .99])
              )

value                                                        \
                  count        mean        std    min         25%         50%   
label                                                                           
DiasBP         920800.0   59.926661  14.408401   1.00   50.000000   58.000000   
Glucose        231947.0  143.183891  63.388926   1.00  105.000000  129.000000   
HeartRate      955108.0   88.350206  19.105930   0.35   75.000000   87.000000   
MeanArtPress   933617.0   77.557488  16.860179   0.20   67.000000   76.000000   
RespRate      1043762.0   20.538296   6.080251   0.17   16.000000   20.000000   
SpO2           929932.0   96.804971   3.675938   1.00   95.000000   97.000000   
SysBP          921226.0  118.676979  23.500270   0.35  102.000000  116.000000   
TempC          255419.0   37.011743   0.899131  22.00   36.444444   36.999999   

                                                               
                     75%         95%         99%          max  
label                                                          
DiasBP         68.000000   85.000000  100.000000   277.000000  
Glucose       163.000000  261.000000  384.000000  1746.000000  
HeartRate     100.000000  122.000000  137.000000   285.000000  
MeanArtPress   86.666702  106.000000  123.000000   299.000000  
RespRate       24.000000   31.000000   37.000000    69.000000  
SpO2           99.000000  100.000000  100.000000   100.000000  
SysBP         133.000000  161.000000  181.000000   325.000000  
TempC          37.599998   38.500002   39.277776    42.777778

time: 3.72 s


In [185]:
#distribution after 120min window aggregate
(vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'value'].notna(), ['label','value']
]
     .groupby(['label'])
         .describe(percentiles=[.25, .5, .75,.95, .99])
              )

value                                                \
                 count        mean        std        min         25%   
label                                                                  
DiasBP        390011.0   64.145827  15.157058   1.000000   54.000000   
Glucose       177409.0  145.931181  62.968776   1.210000  108.000000   
HeartRate     413201.0   89.634256  18.771191   5.600000   76.000000   
MeanArtPress  387947.0   82.101780  17.705805   1.000000   70.666702   
RespRate      464863.0   21.607220   6.156391   1.000000   17.000000   
SpO2          929932.0   96.804971   3.675938   1.000000   95.000000   
SysBP         382796.0  124.710841  22.946955   1.000000  108.000000   
TempC         213809.0   37.013540   0.867512  29.722222   36.444444   

                                                                      
                     50%         75%    95%         99%          max  
label                                                                 
DiasBP         62.000000   73.000000   91.0  108.000000   277.000000  
Glucose       131.000000  166.000000  262.0  384.000000  1746.000000  
HeartRate      88.000000  101.000000  122.0  139.000000   285.000000  
MeanArtPress   80.000000   91.000000  111.0  131.000000   299.000000  
RespRate       21.000000   25.000000   32.0   39.000000    69.000000  
SpO2           97.000000   99.000000  100.0  100.000000   100.000000  
SysBP         122.000000  139.000000  166.0  187.000000   325.000000  
TempC          36.999999   37.555556   38.5   39.222222    42.777778

time: 2.95 s


In [195]:
#distribution after 180min window aggregate
(vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'value'].notna(), ['label','value']
]
     .groupby(['label'])
         .describe(percentiles=[0.01,.25, .5, .75,.95, .99])
              )

value                                               \
                 count        mean        std        min         1%   
label                                                                 
DiasBP        280642.0   66.475506  15.342234   1.000000  38.000000   
Glucose       160858.0  147.314710  63.130020   3.000000  66.000000   
HeartRate     302117.0   91.065358  18.955319  11.000000  55.000000   
MeanArtPress  277745.0   84.740149  18.208219   1.000000  55.000000   
RespRate      360994.0   22.308967   6.154733   2.000000  11.000000   
SpO2          929932.0   96.804971   3.675938   1.000000  86.000000   
SysBP         272635.0  128.145165  22.901158   1.000000  86.000000   
TempC         201191.0   37.011140   0.859539  29.722222  35.166667   

                                                                     \
                     25%         50%         75%    95%         99%   
label                                                                 
DiasBP         56.000000   65.000000   75.000000   93.0  111.000000   
Glucose       109.000000  133.000000  167.000000  263.0  385.000000   
HeartRate      78.000000   90.000000  103.000000  124.0  141.000000   
MeanArtPress   73.000000   82.000000   94.000000  114.0  136.000000   
RespRate       18.000000   22.000000   26.000000   33.0   40.000000   
SpO2           95.000000   97.000000   99.000000  100.0  100.000000   
SysBP         111.000000  125.000000  142.000000  169.0  190.000000   
TempC          36.444444   36.944444   37.555555   38.5   39.222222   

                           
                      max  
label                      
DiasBP         277.000000  
Glucose       1746.000000  
HeartRate      285.000000  
MeanArtPress   299.000000  
RespRate        69.000000  
SpO2           100.000000  
SysBP          325.000000  
TempC           42.777778

time: 1.16 s


In [ ]:
(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'icustay_id']==233004.0) &
                                 (vitals_all_nosummary_nonull.loc[:,'label']=='MeanArtPress')&
                                 (vitals_all_nosummary_nonull.loc[:,'value']<10.0),'value'])=np.nan

#this cutoff of 10 was picked sort of arbitrarily. ask nelson
(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='MeanArtPress')&
                                 (vitals_all_nosummary_nonull.loc[:,'value']<10.0),'value'])=np.nan

In [ ]:
###need to remove more potentially erroneous values still. 

In [ ]:
pd.DataFrame(vitals_all_nosummary_nonull).to_csv(Path(
    wd+'/data/processed/%s_vitals_prepped.csv' %(date)))


# labs -

In [35]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 4.45 ms


In [36]:
lab_uom= uom_sql_import(Path(wd+'/data/raw/clinical_var_sql/unit_of_mesurement/labs_uom.sql'))
labs_all_nosummary_72 = pd.merge(labs_all_nosummary_72, lab_uom, left_on='label', right_on='label')


del(lab_uom)
labs_all_nosummary_72= labs_all_nosummary_72.rename(
    columns={'valuenum':'value'}) #changing valuenum to value


time: 22.3 s


In [37]:
labs_all_nosummary_72.groupby('label')['uom'].value_counts() #looks good

label        uom    
ALBUMIN      g/dL       10989
ANION GAP    mEq/L      63336
BANDS        %           3570
BICARBONATE  mEq/L      64298
BILIRUBIN    mg/dL      18218
BUN          mg/dL      64765
CHLORIDE     mEq/L      70621
CREATININE   mg/dL      65037
GLUCOSE      mg/dL      83862
HEMATOCRIT   %          80518
HEMOGLOBIN   g/dL       66677
LACTATE      mmol/L     34942
PLATELET     K/uL       59711
POTASSIUM    mEq/L      88479
PT           SECONDS    40594
PTT          sec        42759
SODIUM       mEq/L      73756
WBC          K/uL       58006
Name: uom, dtype: int64

time: 213 ms


In [ ]:
#outliers

In [39]:
#summary value removal
(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==261887) & 
    (labs_all_nosummary_72.loc[:,'label']=='CHLORIDE')
    & (labs_all_nosummary_72.loc[:,'value']==3.4),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==236290) & 
    (labs_all_nosummary_72.loc[:,'label']=='CHLORIDE')
    & (labs_all_nosummary_72.loc[:,'value']==11.0),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==292769) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==28.1),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==298457) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==48.8),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==234174) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==48.7),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==290264) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==42.0),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==290264) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==22.8),'value'])=np.nan

time: 560 ms


In [156]:
labs_all_nosummary_72= labs_all_nosummary_72.loc[labs_all_nosummary_72.loc[:,'value'].notnull(),:]


time: 186 ms


In [157]:
labs_all_nosummary_72

,subject_id,hadm_id,icustay_id,charttime,label,value,t_0,delta,uom
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000,mEq/L
1,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,2149-11-13,0 days 03:20:00.000000000,mEq/L
2,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,2149-11-13,0 days 16:04:00.000000000,mEq/L
3,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,2149-11-13,0 days 19:30:00.000000000,mEq/L
4,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,2149-11-13,1 days 01:48:00.000000000,mEq/L
5,9,150750,220597,2149-11-14 06:00:00,SODIUM,137.0,2149-11-13,1 days 06:00:00.000000000,mEq/L
6,19,109235,273430,2108-08-06 01:08:00,SODIUM,138.0,2108-08-06,0 days 01:08:00.000000000,mEq/L
7,19,109235,273430,2108-08-07 05:07:00,SODIUM,139.0,2108-08-06,1 days 05:07:00.000000000,mEq/L
8,19,109235,273430,2108-08-08 05:20:00,SODIUM,141.0,2108-08-06,2 days 05:20:00.000000000,mEq/L
9,21,111970,216859,2135-02-05 04:48:00,SODIUM,134.0,2135-02-05,0 days 04:48:00.000000000,mEq/L


time: 28.3 ms


In [41]:
# pd.DataFrame(labs_all_nosummary_72).to_csv(Path(
#     wd+'/data/processed/%s_labs_prepped.csv' %(date)))

time: 792 µs


## vent category -

In [42]:
ventcategory_df['uom']='mech/O2/none category'
#ventcategory_df['dtype']= 'factor'
ventcategory_df.head()

,icustay_id,t_0,vent_day1,vent_day2,vent_day3,uom
0,267059.0,2182-11-12,Mech,None,None,category
1,278729.0,2159-11-08,Oxygen,Mech,None,category
2,219670.0,2190-08-01,Oxygen,None,None,category
3,295185.0,2154-11-11,Oxygen,None,None,category
4,211472.0,2193-12-11,None,None,None,category


time: 18.2 ms


In [150]:
ventcategory_df

,icustay_id,t_0,vent_day1,vent_day2,vent_day3,uom
0,267059.0,2182-11-12,Mech,None,None,category
1,278729.0,2159-11-08,Oxygen,Mech,None,category
2,219670.0,2190-08-01,Oxygen,None,None,category
3,295185.0,2154-11-11,Oxygen,None,None,category
4,211472.0,2193-12-11,None,None,None,category
5,259012.0,2120-05-24,None,None,None,category
6,231735.0,2186-04-13,Oxygen,Oxygen,None,category
7,252135.0,2166-04-13,Oxygen,Oxygen,Oxygen,category
8,293255.0,2167-10-21,Oxygen,None,None,category
9,261279.0,2167-10-26,None,None,None,category


time: 22.2 ms


In [151]:
#melting value columns into long format
ventcategory_df= ventcategory_df.melt(
    id_vars=['icustay_id','uom','t_0'],
    var_name='label',
    value_name='value')

#adding a delta column
ventcategory_df['delta']= np.nan
ventcategory_df.loc[ventcategory_df['label']=='vent_day1','delta']= pd.to_timedelta(0,'days')
ventcategory_df.loc[ventcategory_df['label']=='vent_day2','delta']= pd.to_timedelta(1,'days')
ventcategory_df.loc[ventcategory_df['label']=='vent_day3','delta']= pd.to_timedelta(2,'days')

time: 71.2 ms


In [155]:
ventcategory_df.head()

,icustay_id,uom,t_0,label,value,delta
0,267059.0,category,2182-11-12,vent_day1,Mech,0 days 00:00:00
1,278729.0,category,2159-11-08,vent_day1,Oxygen,0 days 00:00:00
2,219670.0,category,2190-08-01,vent_day1,Oxygen,0 days 00:00:00
3,295185.0,category,2154-11-11,vent_day1,Oxygen,0 days 00:00:00
4,211472.0,category,2193-12-11,vent_day1,None,0 days 00:00:00


time: 9.19 ms


In [43]:
# pd.DataFrame(ventcategory_df).to_csv(Path(
#     wd+'/data/processed/%s_ventcategory_prepped.csv' %(date)))

time: 711 µs


## weight and height firstday -
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [44]:
weightfirstday_df['uom']='kg'

time: 1.89 ms


In [45]:
weightfirstday_df.head()  
#weight column is the conglomerate of weight_admin>weight_daily> weight_echoinhosp> weight_echoprehosp

,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp,uom
7,200012,51.2,51.200001,51.200001,NaN,NaN,kg
8,200014,62.0,62.000000,NaN,NaN,NaN,kg
10,200019,65.0,65.000000,NaN,NaN,NaN,kg
16,200030,113.6,113.599998,113.599998,NaN,NaN,kg
18,200033,74.0,74.000000,NaN,NaN,NaN,kg


time: 12.8 ms


In [144]:
weightfirstday_df= weightfirstday_df[weightfirstday_df['weight'].notnull()]
weightfirstday_df= weightfirstday_df[['icustay_id','weight','uom']]

time: 2.85 ms


In [46]:
# pd.DataFrame(weightfirstday_df).to_csv(Path(
#     wd+'/data/processed/%s_weight_prepped.csv' %(date)))

time: 741 µs


In [47]:
heightfirstday_df['uom']='cm'

time: 1.78 ms


In [147]:
heightfirstday_df= heightfirstday_df[heightfirstday_df['height'].notnull()]
heightfirstday_df= heightfirstday_df[['icustay_id','height','uom']]

time: 4.15 ms


In [48]:
# pd.DataFrame(heightfirstday_df).to_csv(Path(
#     wd+'/data/processed/%s_height_prepped.csv' %(date)))

time: 602 µs


# UTI

In [51]:
uti_all_72['value'].unique()#seems good

array(['NEG', 'MOD', 'TR', 'POS', 'SM', 'LG', 'SMALL', ' '], dtype=object)

time: 10.1 ms


In [52]:
uti_all_72_pos= uti_all_72.loc[(uti_all_72.loc[:,'value']!='NEG')&
               uti_all_72.loc[:,'value'].notnull(),:]
uti_all_72_pos= uti_all_72_pos.drop_duplicates(subset=['hadm_id','value','charttime'])

time: 43.7 ms


In [135]:
###coding any positive leukocyte or nitrate between day 1,2 or 3 for all patients. if time granularity is later needed for models, need to revisit this data to get the exact delta value

def uti_categorization(final_pt_df2, uti_all_72_pos):
    
    uti_df_3dayannotation= final_pt_df2[['subject_id','hadm_id','t_0']]
    uti_df_3dayannotation['uti_day1']='Neg/Not_tested'
    uti_df_3dayannotation['uti_day2']='Neg/Not_tested' 
    uti_df_3dayannotation['uti_day3']='Neg/Not_tested' 

    pos_1= uti_all_72_pos.loc[uti_all_72_pos['delta'].between('0 days','1 days'),'hadm_id'].tolist()
    pos_2= uti_all_72_pos.loc[uti_all_72_pos['delta'].between('1 days','2 days'),'hadm_id'].tolist()
    pos_3= uti_all_72_pos.loc[uti_all_72_pos['delta'].between('2 days','3 days'),'hadm_id'].tolist()

    uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(pos_1),'uti_day1']='Pos'
    uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(pos_2),'uti_day2']='Pos'
    uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(pos_3),'uti_day3']='Pos'
    
    return(uti_df_3dayannotation)

time: 14.6 ms


In [136]:
uti_categorization= uti_categorization(final_pt_df2, uti_all_72_pos)

time: 222 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [138]:
#melting value columns into long format
uti_categorization= uti_categorization.melt(
    id_vars=['subject_id','hadm_id','t_0'],
    var_name='label',
    value_name='value')

#adding a delta column
uti_categorization['delta']= np.nan
uti_categorization.loc[uti_categorization['label']=='uti_day1','delta']= pd.to_timedelta(0,'days')
uti_categorization.loc[uti_categorization['label']=='uti_day2','delta']= pd.to_timedelta(1,'days')
uti_categorization.loc[uti_categorization['label']=='uti_day3','delta']= pd.to_timedelta(2,'days')

time: 44.3 ms


In [149]:
uti_categorization['uom']='pos/neg category'

time: 1.93 ms


In [55]:
## pd.DataFrame(uti_all_72_notnull).to_csv(Path(
##     wd+'/data/processed/%s_uti_all_72_notnull.csv' %(date)))

# pd.DataFrame(uti_categorization).to_csv(Path(
#     wd+'/data/processed/%s_uti_prepped.csv' %(date)))

time: 864 µs


# bloodgas

In [102]:
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]
bg_all_nosummary_72 = bg_all_nosummary_72.rename(index=str, columns={'valueuom':'uom'})
bg_all_nosummary_72= bg_all_nosummary_72.loc[~(bg_all_nosummary_72.loc[:,'value']=='.'),:]

#may need to remove outliers, haven't done as of 10/22/18
bg_all_nosummary_72.head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
82,9,150750,220597,2149-11-13 03:34:00,BASEEXCESS,NaN,-1,mEq/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
83,9,150750,220597,2149-11-13 03:34:00,CALCIUM,1.05,1.05,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
84,9,150750,220597,2149-11-13 03:34:00,LACTATE,1.80,1.8,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
85,9,150750,220597,2149-11-13 03:34:00,PCO2,34.00,34,mm Hg,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
86,9,150750,220597,2149-11-13 03:34:00,PH,7.42,7.42,units,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000


time: 883 ms


#### erroneous value removal

NOTE: these are hard coded to positioning. if anything in the bg_all_nosummary is changed than these positions need to be updated. 

In [103]:
##calcium
#fixing the calcium errors w/o hard coding
(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==249571) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'valuenum']==94.00),'valuenum'])=0.94#.where('valuenum'==94.00))

(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==249571) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'value']=='094'),'value'])=0.94#.where('valuenum'==94.00))


(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==219600) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'valuenum']==97.00),'valuenum'])=0.97#.where('valuenum'==94.00))

(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==219600) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'value']=='097'),'value'])=0.97#.where('valuenum'==94.00))


time: 456 ms


In [104]:
##min chloride
#converting it to a null value without hard coding
(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==261887.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==3.4),'value'])=np.nan
#converting it to a null value without hard coding
(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==261887.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==3.4),'valuenum'])=np.nan

#changing the values without hard coding. 
(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==236290.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==11.0),'valuenum'])=np.nan

(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==236290.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==11.0),'value'])=np.nan

time: 285 ms


In [106]:
#peep changes summary: 
bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='PEEP')&
                        (bg_all_nosummary_72['valuenum']>38),'valuenum']=np.nan #remove this or set to 50?

time: 52.2 ms


In [ ]:
#temp changes summary:

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==253821)&
                        (bg_all_nosummary_72['valuenum']==18.9),
                        'value']= np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==253821)&
                        (bg_all_nosummary_72['valuenum']==18.9),
                        'valuenum']= np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==251788)&
                        (bg_all_nosummary_72['valuenum']==10.0),
                        'value']= np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==251788)&
                        (bg_all_nosummary_72['valuenum']==10.0),
                        'valuenum']= np.nan

In [59]:
#fio2 changes summary:

##converting a few values to null, thus removing them from the dataset
bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='FIO2') &
                        (bg_all_nosummary_72.loc[:,'value']=='0'),'value']=np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='FIO2') &
                        (bg_all_nosummary_72.loc[:,'value']=='-'),'value']=np.nan


##removing all fio2 values between 1-20.9

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(1.0,20.9)),'valuenum']=np.nan


##values between 0-1 were found to be ratios, not %, so converting these to % 

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(1.0,20.9)),'valuenum']=np.nan

fio2_dec= bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(0.0,1.0)),'valuenum']

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(0.0,1.0)),'valuenum'] = fio2_dec *100

del(fio2_dec)

time: 21.7 ms


#### removing null values annotated abov

In [154]:
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]

KeyError: 'value'

time: 36.2 ms


### splitting specimen out for vent vs non-vent bg data

In [62]:
#adding specimen tag to filter only arterial samples for vent data. 
specimen_df= bg_all_nosummary_72.loc[bg_all_nosummary_72.loc[:,'label']=='SPECIMEN',['unique_var','label','value']]#unique()
specimen_df=specimen_df.rename(index=str, columns={'value':'specimen'})
specimen_df=specimen_df.loc[specimen_df.loc[:,"specimen"]=='ART',:]

bg_all_nosummary_72= pd.merge(bg_all_nosummary_72,specimen_df[['unique_var','specimen']], left_on='unique_var', right_on='unique_var', how='left')
bg_ART_nosummary_72=bg_all_nosummary_72.loc[bg_all_nosummary_72.loc[:,'specimen']=='ART',:]
del(specimen_df)

time: 681 ms


In [87]:
bg_labels=['PH','LACTATE','CALCIUM','TEMPERATURE','POTASSIUM',
           'GLUCOSE','HEMOGLOBIN','SODIUM','CHLORIDE','BICARBONATE']
bg_vent_labels=['PCO2','PaO2','PO2','FIO2','PEEP','O2FLOW']

time: 1.56 ms


In [109]:
#restricting to tests that were chosen to be analysed based on %missingness and clinical relevance
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72.loc[:,'label'].isin(bg_labels),:]
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label'].isin(bg_vent_labels),:]


time: 70.4 ms


In [111]:
#value = valuenum for all specified bg tests. removing unneeded cols

bg_col=['subject_id','hadm_id','icustay_id','charttime','delta','t_0','label','valuenum','uom']
bg_ART_nosummary_72= bg_ART_nosummary_72[bg_col]
bg_all_nosummary_72= bg_all_nosummary_72[bg_col]
del(bg_col)

time: 19.7 ms


In [63]:
# pd.DataFrame(bg_all_nosummary_72).to_csv(Path(
#     wd+'/data/processed/%s_bg_all_prepped.csv' %(date)))

# pd.DataFrame(bg_all_nosummary_72).to_csv(Path(
#     wd+'/data/processed/%s_bg_ART_prepped.csv' %(date)))

time: 1.03 ms


# RRT

In [64]:
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]
rrt_merged_allpt_df_notnull['uom']='category'
rrt_merged_allpt_df_notnull.head()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,icustay_id,t_0,first_charttime,rrt,uom
7,252135.0,2166-04-13,2166-04-13 00:00:00,1.0,category
8,293255.0,2167-10-21,2167-10-21 04:31:00,1.0,category
9,261279.0,2167-10-26,2167-10-26 01:00:00,1.0,category
20,251526.0,2129-12-20,2129-12-20 09:32:00,1.0,category
31,255314.0,2181-10-11,2181-10-11 01:00:00,1.0,category


time: 76.7 ms


In [79]:
rrt_merged_allpt_df_notnull['delta']=pd.to_timedelta(
    pd.to_datetime(rrt_merged_allpt_df_notnull['first_charttime'])-
    pd.to_datetime(rrt_merged_allpt_df_notnull['t_0']),
    'days')

time: 47.3 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [80]:
rrt_merged_allpt_df_notnull.head()

,icustay_id,t_0,first_charttime,rrt,uom,delta
7,252135.0,2166-04-13,2166-04-13 00:00:00,1.0,category,00:00:00
8,293255.0,2167-10-21,2167-10-21 04:31:00,1.0,category,04:31:00
9,261279.0,2167-10-26,2167-10-26 01:00:00,1.0,category,01:00:00
20,251526.0,2129-12-20,2129-12-20 09:32:00,1.0,category,09:32:00
31,255314.0,2181-10-11,2181-10-11 01:00:00,1.0,category,01:00:00


time: 9.46 ms


In [65]:
# pd.DataFrame(rrt_merged_allpt_df_notnull).to_csv(Path(
#     wd+'/data/processed/%s_rrt_prepped.csv' %(date)))

time: 603 µs


# GCS_72

In [66]:
gcs72_df['uom']='GCS_score'

time: 2.92 ms


In [67]:
list(gcs72_df)

['subject_id',
 'hadm_id',
 'icustay_id',
 'day',
 'mingcs',
 'gcsmotor',
 'gcsverbal',
 'gcseyes',
 'endotrachflag',
 'ICU_admit',
 't_0',
 'admit_plus_day',
 'delta',
 'uom']

time: 3.38 ms


In [68]:
gcs72_df=gcs72_df[['subject_id','hadm_id','icustay_id','delta','mingcs','t_0']]

time: 16.1 ms


In [69]:
# pd.DataFrame(gcs72_df).to_csv(Path(
#     wd+'/data/processed/%s_gcs_prepped.csv' %(date)))

time: 482 µs


# SOFA_72
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [70]:
sofa_df_72['uom']='daily_sofa_score'

#adding day delta column
sofa_df_72=sofa_df_72.sort_values(['hadm_id','day','t_0']) #good
sofa_df_72['day_rank']=sofa_df_72.groupby('icustay_id')['day'].rank()
sofa_df_72['delta']=pd.to_timedelta((sofa_df_72['day_rank']-1), 'days')

time: 166 ms


In [71]:
sofa_score_72= sofa_df_72[['subject_id','hadm_id','icustay_id','delta','sofa','t_0','uom']]

time: 10.6 ms


In [72]:
sofa_score_72

,subject_id,hadm_id,icustay_id,delta,sofa,t_0,uom
63554,54610,100003,209281,0 days,4,2150-04-18,daily_sofa_score
36471,23018,100007,217937,0 days,1,2145-03-31,daily_sofa_score
36472,23018,100007,217937,1 days,3,2145-03-31,daily_sofa_score
36473,23018,100007,217937,2 days,1,2145-03-31,daily_sofa_score
670,533,100009,253656,0 days,5,2162-05-17,daily_sofa_score
671,533,100009,253656,1 days,1,2162-05-17,daily_sofa_score
672,533,100009,253656,2 days,2,2162-05-17,daily_sofa_score
71986,68591,100016,217590,0 days,2,2188-05-25,daily_sofa_score
71987,68591,100016,217590,1 days,2,2188-05-25,daily_sofa_score
71988,68591,100016,217590,2 days,0,2188-05-25,daily_sofa_score


time: 47.1 ms


In [73]:
# pd.DataFrame(sofa_score_72).to_csv(Path(
#     wd+'/data/processed/%s_sofa_prepped.csv' %(date)))

time: 824 µs
